In [9]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import os
import glob

from dipy.core.sphere import faces_from_sphere_vertices, unique_edges
from dipy.data import get_sphere, HemiSphere, Sphere
from dipy.io.streamline import load_trk
from dipy.io.streamline import save_trk
from dipy.tracking import utils
from dipy.tracking.local import (CmcTissueClassifier, LocalTracking)
from dipy.tracking.streamline import Streamlines
from dipy.tracking.utils import random_seeds_from_mask, move_streamlines
from dipy.direction import ProbabilisticDirectionGetter, DeterministicMaximumDirectionGetter
from os import path

In [6]:
os.chdir('/space/hemera/1/users/cmaffei/hcp_processing/mgh_1015/')
wd = '/space/hemera/1/users/cmaffei/hcp_processing/mgh_1015/'

In [1]:
from dipy.tracking.distances import approx_polygon_track

In [45]:
bundle, hdr = load_trk('prova_nuova_pve_30_05.trk')
bundle = Streamlines(bundle)
bundle_moved = move_streamlines(bundle, np.eye(4), img.affine)

In [40]:
bundle_downsampled = [approx_polygon_track(s, 0.4) for s in bundle_moved]
bundle_downsampled = Streamlines(bundle_downsampled)

In [41]:
mask=("mask_er.nii.gz")
img = nib.load(mask)
mask = img.get_data()

In [42]:
affine= np.eye(4)
save_trk('prova_downsampled_04.trk',
         bundle_downsampled,
         img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

In [46]:
# Compute lookup table
from dipy.denoise.enhancement_kernel import EnhancementKernel

D33 = 1.0
D44 = 0.02
t = 1
k = EnhancementKernel(D33, D44, t)

# Apply FBC measures
from dipy.tracking.fbcmeasures import FBCMeasures

fbc = FBCMeasures(bundle_moved, k)

# Apply a threshold on the RFBC to remove spurious fibers
fbc_sl_thres, clrs_thres, rfbc_thres = \
  fbc.get_points_rfbc_thresholded(0.125, emphasis=0.01)
    
    
    
save_trk('prova_fbc.trk',
         fbc_sl_thres,
         img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

The kernel already exists. Loading from /tmp/kernel_d33@1.00_d44@0.02_t@1.00_numverts100.npy
The minimum fiber length is 10 points.                     Shorter fibers were found and removed.


Exception ValueError: 'max() arg is an empty sequence' in 'dipy.tracking.fbcmeasures.FBCMeasures.compute' ignored


AttributeError: Memoryview is not initialized